In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.dt_agent import DTAgent
from sklearn.model_selection import train_test_split

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
np.random.seed(SEED)

#### The data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_6.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,12.322384,163.121429,4.121959,-1.00000,433.145097,100.147359,51.037057,3.691276,1,0.819482,147.693827,59.199141,41.958432,20.792161,101.383932,36.967153,11.782901,Unspecified anemia
1,8.298889,-1.000000,2.076950,-1.00000,483.617753,98.431076,-1.000000,2.529350,0,-1.000000,-1.000000,-1.000000,36.118322,-1.000000,-1.000000,24.896668,-1.000000,Hemolytic anemia
2,12.696391,3.393723,-1.000000,-1.00000,451.933132,79.486542,85.001345,4.791902,1,-1.000000,4.852168,89.831485,44.946238,0.965963,-1.000000,38.089174,18.808390,Iron deficiency anemia
3,12.705102,-1.000000,2.305379,-1.00000,-1.000000,81.057541,135.371313,4.702253,1,1.324140,32.717943,76.524319,-1.000000,27.439316,-1.000000,38.115305,-1.000000,Aplastic anemia
4,8.211543,29.622561,-1.000000,0.93619,479.914773,78.388480,-1.000000,3.142634,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,24.634629,-1.000000,Iron deficiency anemia


In [4]:
utils.get_dt_performance(df)

(0.6596428571428572,
 0.6536656960257498,
 0.8005238687826254,
 datetime.timedelta(microseconds=4001))

In [5]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train_df), np.array(y_train_df)
X_test, y_test = np.array(X_test_df), np.array(y_test_df)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [6]:
y_test[:5]

array([1, 5, 3, 2, 0], dtype=int64)

#### The DT agent

In [7]:
%%time
dt_agent = DTAgent(X_test, y_test)
test_df = dt_agent.test()
test_df.head()

Testing done.....
Wall time: 2min 32s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Vitam...",1.0,1.0
1,4.0,1.0,0.0,-1.0,0.0,"[hemoglobin, mcv, ret_count, Aplastic anemia]",5.0,6.0
2,6.0,2.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, ferritin, tibc, Anem...",3.0,3.0
3,4.0,3.0,0.0,-1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Incon...",2.0,7.0
4,4.0,4.0,0.0,-1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Incon...",0.0,7.0


In [8]:
# %%time
# dt_agent.test_sample(2)

In [9]:
# from datetime import datetime
# start = datetime.now()
# dt_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [10]:
# X_test_df.head()

In [8]:
len(test_df), len(X_test)

(14000, 14000)

In [9]:
test_df.to_csv('../../test_dfs/many_features/0.1/correlated/dt_test_df_corr_noisy_6.csv', index=False)

In [10]:
# test_df = pd.read_csv('../../test_dfs/many_features/dt_test_df_0.1.csv')
# test_df.head()

In [11]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

69.69999999999999

In [12]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(3.999285714285714, 0.394)

In [13]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.697, 0.6912169884144359, 0.8392229382042525)

In [17]:
# utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [18]:
# utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])